## Assignment - Build a confidence interval

A confidence interval refers to a neighborhood around some point estimate, the size of which is determined by the desired p-value. For instance, we might say that 52% of Americans prefer tacos to burritos, with a 95% confidence interval of +/- 5%.

52% (0.52) is the point estimate, and +/- 5% (the interval $[0.47, 0.57]$) is the confidence interval. "95% confidence" means a p-value $\leq 1 - 0.95 = 0.05$.

In this case, the confidence interval includes $0.5$ - which is the natural null hypothesis (that half of Americans prefer tacos and half burritos, thus there is no clear favorite). So in this case, we could use the confidence interval to report that we've failed to reject the null hypothesis.

But providing the full analysis with a confidence interval, including a graphical representation of it, can be a helpful and powerful way to tell your story. Done well, it is also more intuitive to a layperson than simply saying "fail to reject the null hypothesis" - it shows that in fact the data does *not* give a single clear result (the point estimate) but a whole range of possibilities.

How is a confidence interval built, and how should it be interpreted? It does *not* mean that 95% of the data lies in that interval - instead, the frequentist interpretation is "if we were to repeat this experiment 100 times, we would expect the average result to lie in this interval ~95 times."

For a 95% confidence interval and a normal(-ish) distribution, you can simply remember that +/-2 standard deviations contains 95% of the probability mass, and so the 95% confidence interval based on a given sample is centered at the mean (point estimate) and has a range of +/- 2 (or technically 1.96) standard deviations.

Different distributions/assumptions (90% confidence, 99% confidence) will require different math, but the overall process and interpretation (with a frequentist approach) will be the same.

Your assignment - using the data from the prior module ([congressional voting records](https://archive.ics.uci.edu/ml/datasets/Congressional+Voting+Records)):


### Confidence Intervals:
1. Generate and numerically represent a confidence interval
2. Graphically (with a plot) represent the confidence interval
3. Interpret the confidence interval - what does it tell you about the data and its distribution?

### Chi-squared tests:
4. Take a dataset that we have used in the past in class that has **categorical** variables. Pick two of those categorical variables and run a chi-squared tests on that data
  - By hand using Numpy
  - In a single line using Scipy


In [2]:
import pandas as pd
import numpy as np
import scipy.stats as stats

In [3]:
data='https://archive.ics.uci.edu/ml/machine-learning-databases/voting-records/house-votes-84.data'
col_names=['class', 'handicapped-infants', 'water-project', 'adoption-of-the-budget',
           'physician-fee-freeze', 'el-salvador-aid', 'religious-groups-in-schools',
           'anti-satellite', 'aid-to-nicaraguan-contras', 'mx-missile', 'immigration',
           'synfuels', 'education-spending', 'superfund', 'crime', 'duty-free-exports', 
           'act-south-africa']

votes=pd.read_csv(data, names=col_names)
votes.head()

,class,handicapped-infants,water-project,adoption-of-the-budget,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels,education-spending,superfund,crime,duty-free-exports,act-south-africa
0,republican,n,y,n,y,y,y,n,n,n,y,?,y,y,y,n,y
1,republican,n,y,n,y,y,y,n,n,n,n,n,y,y,y,n,?
2,democrat,?,y,y,?,y,y,n,n,n,n,y,n,y,y,n,n
3,democrat,n,y,y,n,?,y,n,n,n,n,y,n,y,n,n,y
4,democrat,y,y,y,n,y,y,n,n,n,n,y,?,y,y,y,y


In [4]:
votes=votes.replace({'n':0, 'y':1, '?':None})
votes.head()

,class,handicapped-infants,water-project,adoption-of-the-budget,physician-fee-freeze,el-salvador-aid,religious-groups-in-schools,anti-satellite,aid-to-nicaraguan-contras,mx-missile,immigration,synfuels,education-spending,superfund,crime,duty-free-exports,act-south-africa
0,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,0.0,1.0
1,republican,0.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,NaN
2,democrat,NaN,1.0,1.0,NaN,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0
3,democrat,0.0,1.0,1.0,0.0,NaN,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
4,democrat,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,NaN,1.0,1.0,1.0,1.0


In [5]:
xtab1=pd.crosstab(votes['water-project'], votes['el-salvador-aid'])
xtab1

el-salvador-aid,0.0,1.0
water-project,,
0.0,106,84
1.0,78,110


###by hand using numpy

In [6]:
#first let's calculate the expected value
xtab_m=pd.crosstab(votes['water-project'], votes['el-salvador-aid'], margins=True)
xtab_m

el-salvador-aid,0.0,1.0,All
water-project,,,
0.0,106,84,190
1.0,78,110,188
All,184,194,378


In [7]:
xtab_m.iloc[2, 2]

378

In [8]:
row_total=xtab_m.iloc[0:2, 2].values
col_total=xtab_m.iloc[2, 0:2].values

In [20]:
exp_lr=row_total*col_total/xtab_m.iloc[2, 2]
exp_rl=row_total*col_total[::-1]/xtab_m.iloc[2, 2]

In [37]:
expected_vals=np.array([[exp_lr[0], exp_rl[0]], [exp_rl[1], exp_lr[1]]])

In [56]:
np.array((expected_vals-xtab1)**2/expected_vals).sum()

7.735040177524986

###using scipy 

In [18]:
stats.chi2_contingency(xtab1)

(7.173224808235082,
 0.0073999634592171295,
 1,
 array([[92.48677249, 97.51322751],
        [91.51322751, 96.48677249]]))

## Stretch goals:

1. Write a summary of your findings, mixing prose and math/code/results. *Note* - yes, this is by definition a political topic. It is challenging but important to keep your writing voice *neutral* and stick to the facts of the data. Data science often involves considering controversial issues, so it's important to be sensitive about them (especially if you want to publish).
2. Apply the techniques you learned today to your project data or other data of your choice, and write/discuss your findings here.
3. Refactor your code so it is elegant, readable, and can be easily run for all issues.

## Resources

- [Interactive visualize the Chi-Squared test](https://homepage.divms.uiowa.edu/~mbognar/applets/chisq.html)
- [Calculation of Chi-Squared test statistic](https://en.wikipedia.org/wiki/Pearson%27s_chi-squared_test)
- [Visualization of a confidence interval generated by R code](https://commons.wikimedia.org/wiki/File:Confidence-interval.svg)
- [Expected value of a squared standard normal](https://math.stackexchange.com/questions/264061/expected-value-calculation-for-squared-normal-distribution) (it's 1 - which is why the expected value of a Chi-Squared with $n$ degrees of freedom is $n$, as it's the sum of $n$ squared standard normals)